In [10]:
import fastf1
import pandas as pd

session_fp1 = fastf1.get_session(2025, 'Canada', 'FP1')

In [11]:
# load the FP1 data
session_fp1.load()

core           INFO 	Loading data for Canadian Grand Prix - Practice 1 [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '10', '12', '14', '16', '18', 

In [21]:
# create a dataframe for all the FP1 data 
# these are the columns we will be working with
laps_columns_to_keep = [
    "LapTime",
    "LapStartTime",
    "TyreLife",
    "SpeedI1",
    "SpeedI2",
    "SpeedFL",
    "Compound",
    "IsAccurate",
    "Driver",
    "Stint",
    "TrackStatus"
]

laps1 = session_fp1.laps.copy()
laps_fp1 = laps1[laps_columns_to_keep]
laps_fp1


,LapTime,LapStartTime,TyreLife,SpeedI1,SpeedI2,SpeedFL,Compound,IsAccurate,Driver,Stint,TrackStatus
0,0 days 00:01:40.099000,0 days 00:15:30.753000,1.0,249.0,268.0,286.0,SOFT,False,VER,1.0,12
1,0 days 00:01:16.067000,0 days 00:17:07.002000,2.0,253.0,277.0,286.0,SOFT,True,VER,1.0,1
2,0 days 00:02:16.828000,0 days 00:18:23.069000,3.0,103.0,154.0,288.0,SOFT,True,VER,1.0,1
3,0 days 00:01:14.478000,0 days 00:20:39.897000,4.0,257.0,280.0,289.0,SOFT,True,VER,1.0,1
4,0 days 00:02:05.872000,0 days 00:21:54.375000,5.0,110.0,213.0,290.0,SOFT,True,VER,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
545,0 days 00:01:22.262000,0 days 01:10:31.357000,13.0,248.0,194.0,269.0,SOFT,True,BEA,4.0,1
546,0 days 00:01:17.377000,0 days 01:11:53.619000,14.0,253.0,277.0,273.0,SOFT,True,BEA,4.0,1
547,0 days 00:01:16.793000,0 days 01:13:10.996000,15.0,NaN,276.0,275.0,SOFT,True,BEA,4.0,1
548,0 days 00:02:03.949000,0 days 01:14:27.789000,16.0,152.0,249.0,80.0,SOFT,True,BEA,4.0,1


In [22]:
# we also need the weather data for the extensive analysis 
weather_columns_to_keep = [
    "Time",
    "TrackTemp"
]

weather1 = session_fp1.weather_data.copy()
weather_fp1 = weather1[weather_columns_to_keep]
# sd weather_fp1

,Time,TrackTemp
0,0 days 00:00:35.985000,41.4
1,0 days 00:01:36.019000,41.4
2,0 days 00:02:36.014000,41.4
3,0 days 00:03:36.014000,41.4
4,0 days 00:04:36.002000,41.4
...,...,...
75,0 days 01:15:36.122000,43.8
76,0 days 01:16:36.129000,44.0
77,0 days 01:17:36.129000,44.0
78,0 days 01:18:36.136000,44.0


In [23]:
# 
laps_fp1['LapStartTime'] = pd.to_timedelta(laps_fp1['LapStartTime'])
weather_fp1['Time'] = pd.to_timedelta(weather_fp1['Time'])

/var/folders/bc/_h0z1r_n7xsczt7nbblx6sxr0000gn/T/ipykernel_69761/1065818909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_fp1['LapStartTime'] = pd.to_timedelta(laps_fp1['LapStartTime'])
/var/folders/bc/_h0z1r_n7xsczt7nbblx6sxr0000gn/T/ipykernel_69761/1065818909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_fp1['Time'] = pd.to_timedelta(weather_fp1['Time'])


In [24]:
laps_fp1 = laps_fp1.sort_values('LapStartTime')
weather_fp1 = weather_fp1.sort_values('Time')

In [25]:
merged_fp1 = pd.merge_asof(
    laps_fp1,
    weather_fp1,
    left_on='LapStartTime',
    right_on='Time',
    direction='backward'
)

In [26]:
merged_fp1

,LapTime,LapStartTime,TyreLife,SpeedI1,SpeedI2,SpeedFL,Compound,IsAccurate,Driver,Stint,TrackStatus,Time,TrackTemp
0,NaT,0 days 00:14:10.222000,1.0,226.0,248.0,264.0,SOFT,False,OCO,1.0,1,0 days 00:13:36.065000,41.2
1,NaT,0 days 00:14:19.847000,1.0,178.0,256.0,256.0,SOFT,False,PIA,1.0,1,0 days 00:13:36.065000,41.2
2,NaT,0 days 00:14:22.440000,1.0,208.0,253.0,NaN,MEDIUM,False,HUL,1.0,12,0 days 00:13:36.065000,41.2
3,0 days 00:01:41.211000,0 days 00:14:23.675000,1.0,233.0,257.0,281.0,SOFT,False,HAM,1.0,1,0 days 00:13:36.065000,41.2
4,0 days 00:01:46.560000,0 days 00:14:31.331000,1.0,187.0,250.0,279.0,SOFT,False,COL,1.0,1,0 days 00:13:36.065000,41.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,NaT,0 days 01:17:26.538000,10.0,239.0,245.0,NaN,SOFT,False,HUL,7.0,1,0 days 01:16:36.129000,44.0
546,NaT,0 days 01:17:30.270000,16.0,238.0,261.0,NaN,SOFT,False,HAD,5.0,1,0 days 01:16:36.129000,44.0
547,NaT,0 days 01:17:32.035000,15.0,198.0,274.0,NaN,SOFT,False,COL,4.0,1,0 days 01:16:36.129000,44.0
548,NaT,0 days 01:17:35.033000,11.0,170.0,230.0,NaN,SOFT,False,ANT,5.0,1,0 days 01:16:36.129000,44.0
